In [ ]:
import pypsa
import pandas as pd
import numpy as np
import matplotlib as plt

In [ ]:
solver = "highs"

*** 
Check-point 1:

**Build a network in PyPSA with one node (`bus`) named "Utopia" and attach an electricity load (`load`) to it. For simplicity, we assume the load profile to be flat for now. Afterwards, we want to supply electricity by attaching one gas-fired power plant implemented as (`generator`) (you have to call [`network.set_snapshots`](https://pypsa.readthedocs.io/en/latest/api/_source/pypsa.Network.set_snapshots.html) to select a year). As help you should have a look at the [PyPSA documentation](https://pypsa.readthedocs.io/en/latest/index.html) and the [minimal lopf example](https://pypsa.readthedocs.io/en/latest/examples/minimal_example_pf.html), understand what the [components documentation](https://pypsa.readthedocs.io/en/latest/user-guide/components.html) of PyPSA gives you and that you can find the underlying objective function and constraints in the [LOPF documentation](https://pypsa.readthedocs.io/en/latest/user-guide/optimal-power-flow.html).**

> **Remarks:** For time reasons, you do not have to build the network from scratch. However, to get you acquainted with PyPSA we have omitted a few elements or some of the parameters of the network marked by three question marks `???`. Either, you have to add an element similar to the one in the box above or add a few parameters.

<img src="../data/block diagrams/d1_e1-1.svg" width=300/>

Initialize network

In [ ]:
# Create empty PyPSA network
network = pypsa.Network()

In [ ]:
# Set snapshots to the year 2023 and at hourly resolution
snapshots = pd.date_range("01-01-2023", "01-01-2024", freq="h", inclusive="left")
network.set_snapshots(snapshots)

In [ ]:
network.snapshots

Add `Utopia` bus with electricity as `carrier`

In [ ]:
network.add(class_name="Bus", name="electricity")

In [ ]:
network.buses

Add constant hourly electricity load of `100MW`

In [ ]:
network.add(class_name="Load", bus="electricity", name="electricity_load", p_set=100)

In [ ]:
network.loads

Add a gas-fired power plant at the electricity bus with a capacity of `200MW` and marginal cost of `20$/MWh` to the network.

In [ ]:
# Creating a gas-fired power plant
network.add(
    class_name="Generator",
    name="gas_power_plant",
    bus="electricity",
    carrier="gas",
    p_nom=200,  # unit: MW
    marginal_cost=20,  # unit: $/MWh
)

Now try to solve your network

In [ ]:
# Solve network using cbc solver
network.optimize(solver_name=solver)

Lets look at some results! Back to [readthedocs](https://pypsa.readthedocs.io/en/latest/user-guide/components.html#network). How would you look at results?

In [ ]:
network.buses_t.marginal_price.head()

In [ ]:
network.generators_t.p.head()

Ending of check-point 1 - export network

In [ ]:
# Export network
network.export_to_netcdf("../../results/network_d1_e1-1.nc")

***
Check-point 2:

**Remove the generator from the previous exercise and replace it with two actual power plants: one nuclear and one gas-fired. The annualized capital costs and marginal costs for these plants should be calculated using the provided input parameters. Instead of setting the plant capacities exogenously, allow the model to determine the optimal capacities based on the linear optimization problem with power flow (LOPF).**

<img src="../data/block diagrams/d1_e1-2.svg" width=350/>

Beginning

In [ ]:
# Import check-point 1 network
network = pypsa.Network("../../results/network_d1_e1-1.nc")

In [ ]:
# remove generator from previous check-point
network.remove(class_name="Generator", name="gas_power_plant")

In [ ]:
network.generators

In [ ]:
network.buses

Add a nuclear power plant to your network based on the provided capital expenditure (CAPEX), fixed operation & maintenance cost (FOM), variable operation & maintenance cost (VOM), fuel cost, efficiency, interest rate and lifetime.

> **Source:** all costs for the example are taken from [PyPSA's technology database](https://github.com/PyPSA/technology-data/blob/master/outputs/costs_2025.csv).

In [ ]:
# Techno-economic parameters of the nuclear power plant:
lifetime = 40  # years
interest = 0.05  # unit: -
CAPEX = 8594135  # $/MW
FOM = 0.0127 * CAPEX  # $/MW per year (1.27% of CAPEX)
VOM = 3.55  # $/MWh
fuel_cost = 3.41  # $/MWh_th per unit Uranium consumed
efficiency = 0.326

In [ ]:
# We need to calculate annualized capital expenditure
def calculate_annualised_capex(capex: float, interest: float, lifetime: int):
    crf = (
        interest * (1 + interest) ** lifetime / ((1 + interest) ** lifetime - 1)
    )  # Capital recovery factor
    return capex * crf

In [ ]:
annualized_capex = calculate_annualised_capex(CAPEX, interest, lifetime)
network.add(
    class_name="Generator",
    name="nuclear_power_plant",
    bus="electricity",
    marginal_cost=VOM + fuel_cost / efficiency,  # $/MWh
    capital_cost=annualized_capex + FOM,  # $/MW
    p_nom_extendable=True,  # Allow endogenous investment
    efficiency=efficiency,
)

In [ ]:
network.generators

Add a gas-fired power plant to your network

In [ ]:
# Techno-economic parameters of the gas-fired combined cycle power plant (CCGT):
lifetime = 25  # years
interest = 0.05  # unit: -
CAPEX = 904779  # $/MW
FOM = 0.0334 * CAPEX  # $/MW per year (3.34% of CAPEX)
VOM = 4.55  # $/MWh variable
fuel_cost = 24.57  # $/MWh_th per unit gas consumed
efficiency = 0.57

In [ ]:
annualized_capex = calculate_annualised_capex(CAPEX, interest, lifetime)

network.add(
    class_name="Generator",
    name="gas_power_plant",
    bus="electricity",
    marginal_cost=VOM + fuel_cost / efficiency,  # $/MWh
    capital_cost=annualized_capex + FOM,  # $/MW
    p_nom_extendable=True,  # Allow endogenous investment
    efficiency=efficiency,
)

In [ ]:
network.generators

Ending of check-point 2 - Solve network, analyse results and export network

In [ ]:
# Solve network using cbc solver
network.optimize(solver_name=solver)

Look at your results! 

In [ ]:
network.generators["p_nom_opt"]

In [ ]:
network.generators_t.p.head()

In [ ]:
# Export network
network.export_to_netcdf("../../results/network_d1_e1-2.nc")

***
Check-point 3:

**Add a Variable Renewable Energy (VRE) power plant to the network: Solar PV with extendable capacity.**

**Hint:** Solar PV can only provide electricity during day time and its generation pattern depends on solar irradiation.

<img src="../data/block diagrams/d1_e1-3.svg" width=500/>

In [ ]:
# Import check-point 2 network
network = pypsa.Network("../../results/network_d1_e1-2.nc")

In [ ]:
network.generators[["capital_cost", "marginal_cost", "p_nom_extendable"]]

Import an example solar daily availability and assume same pattern throughout the year

In [ ]:
# Import an example daily pattern
solar_pattern = pd.read_csv("../../data/weather data/example_solar_daily_pattern.csv")[
    "daily pattern"
]
# annual time-series availability of solar farm (just a simplified example)
yearly_avail = pd.Series(list(solar_pattern) * 365, index=network.snapshots)

In [ ]:
yearly_avail.head(24).plot()

Add a solar farm to your network

In [ ]:
# Techno-economic parameters of the solar farm:
lifetime = 37.5  # years
interest = 0.05  # unit: -
CAPEX = 676570  # $/MW
FOM = 0.017275 * CAPEX  # $/MW per year (1.7275% of CAPEX)
VOM = 0.0106  # $/MWh
fuel_cost = 0  # per unit of solar irradiation consumed

In [ ]:
annualized_capex = calculate_annualised_capex(
    capex=CAPEX, interest=interest, lifetime=lifetime
)

network.add(
    class_name="Generator",
    name="solar_farm",
    bus="electricity",
    marginal_cost=VOM,
    capital_cost=annualized_capex + FOM,
    p_nom_extendable=True,
    p_max_pu=yearly_avail,  # Solar farm availability
)

Ending of check-point 3: Solve network, analyze results and export network

In [ ]:
# Solve network using cbc solver
network.optimize(solver_name=solver)

In [ ]:
# Inspect solar farm interaction with other power plants to supply loads
network.generators["p_nom_opt"]

In [ ]:
gen = network.generators_t.p
gen[["solar_farm", "gas_power_plant"]].head(50).plot(kind="area", stacked=True)

In [ ]:
# Exporting check-point 3 network
network.export_to_netcdf("../../results/network_d1_e1-3.nc")